In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Tree.tree_warm_start'




In [2]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('As')
params.root_node.street = 2
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))

### 1. Exploitability for original CFR and modified CFR

In [3]:
--- Building CFR Solver
tree100 = builder:build_tree(params)
tree100_cfr = TreeCFR()
tree100_cfr:run_cfr(tree100, starting_ranges,100)
print(tree100.exploitability)

180


In [4]:
--- Building CFR Solver
tree102 = builder:build_tree(params)
tree102_cfr = TreeCFR2()
tree102_cfr:run_cfr(tree102, starting_ranges,100)
print(tree102.exploitability)

180


In [5]:
exploit_cfr100 = tree100_cfr.exploitability_vec
exploit_cfr102 = tree102_cfr.exploitability_vec

In [6]:
local num_iter = torch.range(1,exploit_cfr100:size(1))
local plot = Plot()

plot:line(num_iter,exploit_cfr100,'red' ,'Original')
plot:line(num_iter,exploit_cfr102,'blue' ,'New CFR')
plot:legend(true):title('Exploitability of different strategies, 800 iterations')
plot:xaxis('Scaled number of iterations'):yaxis('Exploitability'):redraw()


### 2. Exploitability for different Strategies

In [7]:
number_of_iterations = 100

In [8]:
--- Get Data from 2000 iterations
tree800 = builder:build_tree(params)
tree_data800 = TreeData(tree800)
tree800_cfr = TreeCFR()
tree800_cfr:run_cfr(tree800, starting_ranges,number_of_iterations)
tree_data800:get_training_set(tree800,1)

### Training the Model

In [9]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model
-- Building trainer:
nn_trainer = NNTrainer(tree_data800,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [10]:
current_criterion = nn.SmoothL1Criterion()
train_loss,test_loss = nn_trainer:train(nn_model,current_criterion,opt,10000)

In [11]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat =nn_trainer.all_data.targets
print(torch.mean(torch.abs(pred_strat- true_strat)))

0.022816972030948


### No Warm Start

In [12]:
--- Building CFR Solver
treeNoWarm = builder:build_tree(params)
treeNoWarm_cfr = TreeCFR2()
treeNoWarm_cfr:run_cfr(treeNoWarm, starting_ranges,number_of_iterations)

### Neural Net Start 

In [13]:
--- NNWarm Starting
treeNNWarm = builder:build_tree(params)
dataNN = TreeData(treeNNWarm)
dataNN:get_training_set(treeNNWarm,1)
dataNN:generate_new_initial_strategies(treeNNWarm,nn_trainer)
---Warm start the regrets:
dataNN:warm_start_regrets(treeNNWarm,tree800)
--- Building CFR Solver
treeNNWarm_cfr = TreeCFR2()
treeNNWarm_cfr:run_cfr(treeNNWarm, starting_ranges,number_of_iterations)

In [14]:
---tree_warm_start = TreeWarmStart()
---tree_warm_start:run_warm_start(treeNNWarm, starting_ranges,80)

In [15]:
--- NNWarm Starting
treeNN_NoRegWarm = builder:build_tree(params)
dataNN_NoReg = TreeData(treeNN_NoRegWarm)
dataNN_NoReg:get_training_set(treeNN_NoRegWarm,1)
dataNN_NoReg:generate_new_initial_strategies(treeNN_NoRegWarm,nn_trainer)
---Warm start the regrets:
---dataNN:warm_start_regrets(treeNN_NoRegWarm,tree800)
--- Building CFR Solver
treeNN_NoRegWarm_cfr = TreeCFR2()
treeNN_NoRegWarm_cfr:run_cfr(treeNN_NoRegWarm, starting_ranges,number_of_iterations)

### Target Warm Start 

In [16]:
--- Target Warm Starting
treeTarRegWarm = builder:build_tree(params)
dataTargReg = TreeData(treeTarRegWarm)
---dataTarg:get_training_set(treeTargWarm,1)
---dataTarg:warm_start_targets(treeTargWarm,tree800)
---dataTarg:warm_start_regrets(treeTargWarm,tree800)
---tree_warm_start = TreeWarmStart()
---tree_warm_start:run_warm_start(treeTargWarm, starting_ranges,number_of_iterations)
dataTargReg:warm_start_targets_and_regrets(treeTarRegWarm,tree800)
--- Building CFR Solver
treeTarRegWarm_cfr = TreeCFR2()
treeTarRegWarm_cfr:run_cfr(treeTarRegWarm, starting_ranges,number_of_iterations)

In [17]:
--- Target Warm Starting
treeTarWarm = builder:build_tree(params)
dataTarg = TreeData(treeTarWarm)
dataTarg:warm_start_targets(treeTarWarm,tree800)
--- Building CFR Solver
treeTarWarm_cfr = TreeCFR2()
treeTarWarm_cfr:run_cfr(treeTarWarm, starting_ranges,number_of_iterations)

In [18]:
--- Target Warm Starting
treeRegWarm = builder:build_tree(params)
dataReg = TreeData(treeRegWarm)
dataReg:warm_start_regrets(treeRegWarm,tree800)
--- Building CFR Solver
treeRegWarm_cfr = TreeCFR2()
treeRegWarm_cfr:run_cfr(treeRegWarm, starting_ranges,number_of_iterations)

### Plot different starts

In [19]:
exploit_unif = tree800_cfr.exploitability_vec
exploit_NoW = treeNoWarm_cfr.exploitability_vec
exploit_NNW = treeNNWarm_cfr.exploitability_vec
---exploit_NN_NoReg = treeNN_NoRegWarm_cfr.exploitability_vec
exploit_TarRegW = treeTarRegWarm_cfr.exploitability_vec
exploit_TarW = treeTarWarm_cfr.exploitability_vec
exploit_RegW = treeRegWarm_cfr.exploitability_vec

In [20]:
local num_iter = torch.range(1,exploit_NoW:size(1))
local plot = Plot()

---plot:line(num_iter,exploit_unif,'black' ,'Original CFR in the paper')
plot:line(num_iter,exploit_NoW,'red' ,'No Warm start CFR = Uniform warm start')
plot:line(num_iter,exploit_NNW,'blue' ,'Neural Net warm start CFR')
---plot:line(num_iter,exploit_NNW,'pink' ,'Neural Net warm start CFR no Regrets')
plot:line(num_iter,exploit_TarRegW,'green' ,'Warm start target and regrets')
plot:line(num_iter,exploit_TarW,'orange' ,'Warm start only targets')
---plot:line(num_iter,exploit_RegW,'pink' ,'Warm start with target values')
plot:legend(true):title('Exploitability of different strategies, 800 iterations')
plot:xaxis('Scaled number of iterations'):yaxis('Exploitability'):redraw()


### Graphical Comparaison

### 3. Comparaision with 2000 CFR2

In [21]:
--- Get Data from 2000 iterations
tree802 = builder:build_tree(params)
tree_data802 = TreeData(tree802)
tree802_cfr = TreeCFR2()
tree802_cfr:run_cfr(tree802, starting_ranges,number_of_iterations)
tree_data802:get_training_set(tree802,1)

In [22]:
-- Building the neural net model
strategy_nn2 = StrategyNN()
nn_model2 = strategy_nn2.model
-- Building trainer:
nn_trainer2 = NNTrainer(tree_data802,nn_model2)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [23]:
current_criterion2 = nn.SmoothL1Criterion()
train_loss,test_loss = nn_trainer:train(nn_model2,current_criterion2,opt,10000)

In [24]:
--- No Warm Start
treeNoWarm2 = builder:build_tree(params)
treeNoWarm_cfr2 = TreeCFR2()
treeNoWarm_cfr2:run_cfr(treeNoWarm2, starting_ranges,number_of_iterations)

In [25]:
--- NNWarm Starting
treeNNWarm2 = builder:build_tree(params)
dataNN2 = TreeData(treeNNWarm2)
dataNN2:get_training_set(treeNNWarm2,1)
dataNN2:generate_new_initial_strategies(treeNNWarm2,nn_trainer2)

In [26]:
--- NNWarm CFR Solver
treeNNWarm_cfr2 = TreeCFR2()
treeNNWarm_cfr2:run_cfr(treeNNWarm2, starting_ranges,number_of_iterations)

In [27]:
--- Target Warm Starting
treeTargWarm2 = builder:build_tree(params)
dataTarg2 = TreeData(treeTargWarm2)
dataTarg2:get_training_set(treeTargWarm2,1)
dataTarg2:warm_start_targets(treeTargWarm2,tree802)

In [28]:
--- Building CFR Solver
treeTargWarm_cfr2 = TreeCFR2()
treeTargWarm_cfr2:run_cfr(treeTargWarm2, starting_ranges,number_of_iterations)

In [29]:
exploit_unif2 = tree802_cfr.exploitability_vec
exploit_NoW2 = treeNoWarm_cfr2.exploitability_vec
exploit_NNW2 = treeNNWarm_cfr2.exploitability_vec
exploit_TargW2 = treeTargWarm_cfr2.exploitability_vec

In [30]:
local num_iter = torch.range(1,exploit_NoW:size(1))
local plot = Plot()

plot:line(num_iter,exploit_unif2,'black' ,'Original CFR in the paper')
plot:line(num_iter,exploit_NoW2,'red' ,'No Warm start CFR')
plot:line(num_iter,exploit_NNW2,'blue' ,'Neural Net warm start CFR')
plot:line(num_iter,exploit_TargW2,'green' ,'Warm start with target values')
plot:legend(true):title('Exploitability of different strategoes')
plot:xaxis('Scaled number of iterations'):yaxis('Exploitability'):redraw()
